# Installations

In [37]:
! pip install langchain_community
! pip install tiktoken
! pip install chromadb
! pip install langchain
! pip install nemoguardrails
! pip install pypdf
! pip install openai
! pip install langchain-openai
! pip install langchain_chroma
! pip install langchain-ollama

In [38]:
from importlib.metadata import version
from tabulate import tabulate

# List of package names
packages = [
    "langchain_community",
    "tiktoken",
    "chromadb",
    "langchain",
    "nemoguardrails",
    "pypdf",
    "openai",
    "langchain-openai",
    "langchain_chroma",
    "langchain-ollama"
]

# Collect package versions
data = []
for package in packages:
    try:
        pkg_version = version(package)
    except Exception as e:
        pkg_version = "Version not found"
    data.append([package, pkg_version])

# Print the formatted table
print(tabulate(data, headers=["Package", "Version"], tablefmt="github"))


| Package             | Version   |
|---------------------|-----------|
| langchain_community | 0.3.19    |
| tiktoken            | 0.9.0     |
| chromadb            | 0.6.3     |
| langchain           | 0.3.20    |
| nemoguardrails      | 0.12.0    |
| pypdf               | 5.4.0     |
| openai              | 1.61.1    |
| langchain-openai    | 0.3.8     |
| langchain_chroma    | 0.2.2     |
| langchain-ollama    | 0.2.3     |


# Loading the data

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = r"/content/Health-insurance.pdf"
loader = PyPDFLoader(file_path)

In [ ]:
docs = loader.load()
docs[0]

Document(metadata={'producer': 'Adobe Acrobat Pro DC (32-bit) 21 Paper Capture Plug-in', 'creator': 'Acrobat PDFMaker 9.0 for Word', 'creationdate': '2011-06-22T12:40:00+05:30', 'author': '', 'company': '', 'moddate': '2021-07-01T02:49:34+05:30', 'sourcemodified': 'D:20110616065908', 'title': 'Terms & condition – Heart Beat                                                                               (A Preliminary Draft for Discussion Purpose only )', 'source': '/content/Health-insurance.pdf', 'total_pages': 25, 'page': 0, 'page_label': '1'}, page_content='Health Companion – Health Insurance Plan  POLICY DOCUMENT \n \n \nPage 1 of 25 \nPolicy Document – Part II \n1. Terms & Conditions \nThe insurance cover provided under this Policy to the Insured Person up to the Sum Insured is and shall \nbe subject to (a) the terms and conditions of this Policy and (b) the receipt of premium, and (c) the \ninformation You provided to Us (including by way of the Proposal or Information Summary Sheet

# Configuring the Guardrails

In [ ]:
yaml_content = """
models:
  - type: main
    engine: ollama
    model: llama3.2-vision:11b

instructions:
  - type: general
    content: |
      You are a bot called the Health Policy Assistant.
      The bot is designed to answer user questions about the Health Insurance Policy.
      The bot is knowledgeable about the terms, conditions, benefits, and exclusions mentioned in the document.
      If the bot does not know the answer to a question, it truthfully says it does not know.
      The bot will only answer questions related to the Health Insurance Policy and will decline politely for off-topic queries.

prompts:
  - task: self_check_input
    content: |
      Your task is to check if the user message is appropriate and relevant to health insurance policy information.

      The user messages:
      - should not contain harmful or abusive language.
      - should not ask the bot to provide false or misleading policy details.
      - should not request personal, sensitive, or confidential information.
      - should not ask for financial advice beyond the scope of the document.
      - should not include explicit, offensive, or unethical content.

      User message: "{{ user_input }}"

      Question: Should the user message be blocked (Yes or No)?
      Answer:

  - task: self_check_output
    content: >
      Model_output: {{ bot_response }}

      Is this output incorrect, misleading, or contains unauthorized financial advice?

      Answer [Yes/No]:

rails:
  input:
    flows:
      - self check input

  output:
    flows:
      - self check output
"""

rag_colang_content = """
# Define allowed user intents
define user greeting
    "hi"
    "hello"
    "how are you"

define bot greeting
    "Hello, I am the Health Policy Assistant. How can I assist you with your health insurance queries?"

define flow greet
    user greeting
    bot greeting

# Define bot response to off-topic questions
define user off topic
    "What do you like?"
    "What is the time?"
    "Tell me a joke?"
    "What is your favorite food?"

define bot response off topic
    "I'm sorry, but I can only provide information related to the Health Insurance Policy."

define flow
    user off topic
    bot response off topic

# Define bot response to policy-related queries
define bot response_refusal
    "I'm sorry, I can't provide that information."

define flow self check input
    $allowed = execute self_check_input
    if not $allowed
        bot response_refusal
        stop

define flow self check output
    $allowed = execute self_check_output
    if not $allowed
        bot response_refusal
        stop

define flow
    user asks about coverage
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

In [ ]:
from nemoguardrails import LLMRails, RailsConfig
config = RailsConfig.from_content(rag_colang_content, yaml_content)

# Using LLM & Embeddings

In [ ]:
from langchain_ollama import OllamaLLM

# Define the base URL where Ollama is hosted
base_url = "http://localhost:11434"

# Load the Ollama LLM model with the specified version
llm = OllamaLLM(
    base_url=base_url,
    model="ollama/llama3.2-vision:11b"
)

In [ ]:
from langchain_ollama import OllamaEmbeddings

# Define the base URL where Ollama is hosted
base_url = "http://localhost:11434"

# Load the specified embedding model
embeddings = OllamaEmbeddings(
    base_url=base_url,
    model="mxbai-embed-large"
)


# Adding Guardrails configuration with LLM

In [ ]:
app = LLMRails(config,llm=llm)

# Injecting the data into the Vector Database

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

# Using PromptTemplate

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """You are a Health Policy Assistant.
Strictly answer the user questions only based on the given contents of the context and do not rely on external knowledge.

- If you don't know the answer or can't find the answer in the context, just say that you don't know, don't try to make up an answer.
- Provide responses that are factually correct and relevant to the health insurance policy.
- Do not give financial or legal advice beyond the policy's stated terms.
- Keep responses concise and accurate.

{context}

{question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

def get_qa_chain(llm):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(),chain_type_kwargs={"prompt":PROMPT},
    )
    return qa_chain

qa_chain = get_qa_chain(app.llm)

app.register_action(qa_chain, name="qa_chain")


# Testing the Guardrails

In [ ]:
user_message=input("Ask your Question: ")
bot_message = await app.generate_async(messages=[{"role": "user", "content": user_message}])
print(bot_message['content'])

Ask your Question: What is your favorite food?
I'm sorry, but I can only provide information related to the Health Insurance Policy.
